# Herbiera

In [2]:
## Import des librairies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import numpy as npx
import keras
import sklearn
import sklearn.metrics
import seaborn
import numpy as np 

2022-12-04 18:04:58.399001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-04 18:04:58.846763: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-04 18:04:58.846796: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-04 18:05:00.327526: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [3]:
## Load Image
def load_image(path):
    img = cv2.imread(path)
    img = cv2.resize(img, (375, 500))
    # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    rotation = cv2.rotate(img, cv2.ROTATE_180)
    flip = cv2.flip(img, 1)
    rotation_flip = cv2.flip(rotation, 1)
    return img, rotation, flip, rotation_flip

## Import des données
def load_images_from_folder(folder):
    train_x, train_y, test_x, test_y = ([], [], [], [])
    aug_train_x, aug_train_y, aug_test_x, aug_test_y = ([], [], [], [])
    count_files = 0
    for _, _, files in os.walk(folder):
        count_files += len(files)
    count = 0
    # Load Train Set
    for specie in os.listdir(folder+'/Train'):
        for image in os.listdir(folder+'/Train/'+specie):
            count += 1
            print('Loading Images: ' + str(count) + '/' + str(count_files))
            img, rotation, flip, rotation_flip = load_image(folder+'/Train/'+specie+'/'+image)
            if img is not None:
                train_x.append(img)
                aug_train_x.append(rotation)
                aug_train_x.append(flip)
                aug_train_x.append(rotation_flip)
                train_y.append(specie)
                aug_train_y.append(specie)
                aug_train_y.append(specie)
                aug_train_y.append(specie)
    # Load Test Set
    for specie in os.listdir(folder+'/Test'):
        for image in os.listdir(folder+'/Test/'+specie):
            count += 1
            print('Loading Images: ' + str(count) + '/' + str(count_files))
            img, rotation, flip, rotation_flip = load_image(folder+'/Test/'+specie+'/'+image)
            if img is not None:
                test_x.append(img)
                aug_test_x.append(rotation)
                aug_test_x.append(flip)
                aug_test_x.append(rotation_flip)
                test_y.append(specie)
                aug_test_y.append(specie)
                aug_test_y.append(specie)
                aug_test_y.append(specie)
    aug_test_x = test_x + aug_test_x
    aug_test_y = test_y + aug_test_y
    aug_train_x = train_x + aug_train_x
    aug_train_y = train_y + aug_train_y
    return train_x, train_y, test_x, test_y, aug_train_x, aug_train_y, aug_test_x, aug_test_y

## Load images
train_x, train_y, test_x, test_y, aug_train_x, aug_train_y, aug_test_x, aug_test_y = load_images_from_folder('dataset')
# aug_train_y, aug_test_x, aug_test_y

Loading Images: 1/330
Loading Images: 2/330
Loading Images: 3/330
Loading Images: 4/330
Loading Images: 5/330
Loading Images: 6/330
Loading Images: 7/330
Loading Images: 8/330
Loading Images: 9/330
Loading Images: 10/330
Loading Images: 11/330
Loading Images: 12/330
Loading Images: 13/330
Loading Images: 14/330
Loading Images: 15/330
Loading Images: 16/330
Loading Images: 17/330
Loading Images: 18/330
Loading Images: 19/330
Loading Images: 20/330
Loading Images: 21/330
Loading Images: 22/330
Loading Images: 23/330
Loading Images: 24/330
Loading Images: 25/330
Loading Images: 26/330
Loading Images: 27/330
Loading Images: 28/330
Loading Images: 29/330
Loading Images: 30/330
Loading Images: 31/330
Loading Images: 32/330
Loading Images: 33/330
Loading Images: 34/330
Loading Images: 35/330
Loading Images: 36/330
Loading Images: 37/330
Loading Images: 38/330
Loading Images: 39/330
Loading Images: 40/330
Loading Images: 41/330
Loading Images: 42/330
Loading Images: 43/330
Loading Images: 44/3

In [4]:
## Afficher 9 images aléatoires
def show_random_images(images, labels):
    fig = plt.figure(figsize=(20, 20))
    for i in range(9):
        ax = fig.add_subplot(3, 3, i+1)
        index = random.randint(0, len(images))
        ax.imshow(images[index])
        ax.set_title(labels[index])
    plt.show()

## Afficher les images
# show_random_images(train_x, train_y)

## Shuffle Dataset
def shuffle(images, labels):
    data = list(zip(list(images), list(labels)))
    random.shuffle(data)
    imgs = [i[0] for i in data]
    lbls = [i[1] for i in data]
    return imgs, lbls

train_x, train_y = shuffle(train_x, train_y)

In [5]:
## Création du modèle
## Find the class of an image using a CNN
model = Sequential()
model.add(Conv2D(4, kernel_size=3, activation='relu', input_shape=(500, 375, 3)))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Conv2D(16, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))

## Compilation du modèle
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['accuracy'])

## Afficher le résumé du modèle
model.summary()

## convertir les labels en vecteurs
def convert_labels(labels):
    new_labels = []
    new_labels_numbers = []
    unique_labels = list(set(labels))
    for label in labels:
        vector = [0] * len(unique_labels)
        vector[unique_labels.index(label)] = 1
        new_labels.append(vector)
        new_labels_numbers.append(unique_labels.index(label))
    return new_labels_numbers
    return new_labels

# def convert_to_numeric(labels):
#     unique = list(set(labels))
#     numeric_labels = []
#     for label in labels:
#         numeric_labels.append([unique.index(label)])
#     return numeric_labels

## Fit du modèle
model.fit(np.array(train_x), np.array(convert_labels(train_y)), epochs=50, batch_size=32,  validation_data=(np.array(test_x), np.array(convert_labels(test_y))))

def analyze_preds(preds, labels):
  confusion_matrix = sklearn.metrics.confusion_matrix(preds,
                                                      labels,
                                                      normalize="true")
  seaborn.heatmap(confusion_matrix,
                  cmap="rocket_r")
  plt.title("Matrice de confusion")
  plt.show()


test_pred = np.argmax(model.predict(np.array(test_x)), axis=-1)
analyze_preds(test_pred, np.argmax(np.array(convert_labels(test_y)), axis=-1))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 498, 373, 4)       112       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 93, 4)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 122, 91, 16)       592       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 30, 22, 16)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 10560)             0         
                                                                 
 dense (Dense)               (None, 64)                6

2022-12-04 18:06:18.175249: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-12-04 18:06:18.175476: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/utility/Herbiera/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-04 18:06:18.175540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/utility/Herbiera/venv/lib/python3.8/site-packages/cv2/../../lib64:
2022-12-04 18:06:18.175585: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.

Epoch 1/50
28/28 [==============================] - 17s 586ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_accuracy: 0.0909
Epoch 2/50
28/28 [==============================] - 15s 524ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_accuracy: 0.0909
Epoch 3/50
28/28 [==============================] - 15s 526ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_accuracy: 0.0909
Epoch 4/50
28/28 [==============================] - 15s 528ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_accuracy: 0.0909
Epoch 5/50
28/28 [==============================] - 15s 552ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_accuracy: 0.0909
Epoch 6/50
28/28 [==============================] - 15s 535ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_accuracy: 0.0909
Epoch 7/50
28/28 [==============================] - 16s 580ms/step - loss: 26.0000 - accuracy: 0.0909 - val_loss: 26.0000 - val_ac